## Data soruce config


In [5]:
import tensorflow as tf
import numpy as np 
import pandas as pd
import matplotlib as mlp
import matplotlib.pyplot as plt
from threading import Lock
import math, json, random, os
import concurrent.futures
AUTO = tf.data.AUTOTUNE

# from tensorboard_model.vision import box_opx, boxutils

In [2]:
DATA_PATH = ""
OUTPUT_DIR = ""
OUTPUT_NAME_PREFIX = 'arthropods_'
IMG_PRE_SHARD = 481
TARGET_WIDTH = 1024

RAW_CALSSES = ['Lepidoptera', 'Hymenoptera', 'Hemiptera', 'Odonata', 'Diptera', 'Araneae', 'Coleoptera',
               '_truncated', '_blurred', '_occluded', ]
CLASSES = [klass for klass in RAW_CALSSES if klass not in ['_truncated', '_blurred', '_occluded',]]

## Dispaly Utills

## Load MetaData

In [8]:
json_filename = os.path.join(DATA_PATH, 'annotations.json')
jpeg_filename = os.path.join(DATA_PATH, "*/*.jpg")

# Read the json file\
def load_json(filename, p): #p represents the path to the json file
    p.step() #p is a progress bar
    with tf.io.gfile.GFile(filename, "r") as f: #tf.io.gfile.GFile is a wrapper around the python file object
        return json.load(f)

def filename_key(filename):
    path, filename = os.path.split(filename) #os.path.split splits the path and the filename
    dirname = os.path.spilt(path)[1] 
    return os.path.join(dirname, filename) 

def load_metadata(filename_p, jpeg_filename): #filename_p is the path to the json file
    json_file = tf.io.gfile.glob(json_filename) #tf.io.gfile.glob returns a list of files that match the given pattern
    jpeg_files = tf.io.gfile.glob(jpeg_filename) 

    p = tf.keras.utils.Progbar(len(json_file)) #tf.keras.utils.Progbar is a progress bar
    with concurrent.futures.ThreadPoolExecutor() as executor: #concurrent.futures.ThreadPoolExecutor is a thread pool executor which is used to execute calls asynchronously
        data = executor.map(lambda x: load_json(x, p), json_file) #executor.map returns an iterator that yields the results of the given function
    data = {filename_key(x['asset']['path']): x for x in data} #filename_key returns the path to the image
    return data, jpeg_files 

In [9]:
RAW_METADATA, JPEG_FILES = load_metadata(json_filename, jpeg_filename)